Offset path generator

In [17]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc
from svgpathtools import parse_path, Line, Path, wsvg

# Sol here https://github.com/mathandy/svgpathtools/blob/fcb648b9bb9591d925876d3b51649fa175b40524/svgpathtools/path.py#L710
def find_intersection(path):
    intersections = []
    for ids, seg in enumerate(path[:-2]): #-2 because next comment
        for ido, other_seg in enumerate(path[ids+2:]): #+2 to ignore the inmediate segment 
            if seg.intersect(other_seg):
                return ids, ids + ido + 1


def offset_curve(path, offset_distance, steps=1000):
    """Takes in a Path object, `path`, and a distance,
    `offset_distance`, and outputs an piecewise-linear approximation 
    of the 'parallel' offset curve."""
    offset_segments = []

    segments = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
    segments.append((path[-1],path[0]))
    print(*segments,sep='\n')

    for idx, (seg, next_seg) in enumerate(segments):
        normal_delta = abs(seg.unit_tangent(1)-next_seg.unit_tangent(0))

        nls = []
        for k in range(steps):
            t = k / steps
            offset_vector = offset_distance * seg.normal(t)
            nl = Line(seg.point(t), seg.point(t) + offset_vector)
            nls.append(nl)
        connect_the_dots = [Line(nls[k].end, nls[k+1].end) for k in range(len(nls)-1)]
        
        if find_intersection(connect_the_dots):
            cut_start,cut_end = find_intersection(connect_the_dots)
            connect_the_dots[cut_start:cut_end+1] = [Line(connect_the_dots[cut_start-1].end, connect_the_dots[cut_end+2].start)]

        offset_segments.extend(connect_the_dots)

        # fix pointy joints #implement determination of large_arc and sweep values  
        if normal_delta > 0.1 and (idx!=len(segments)-1 or path.isclosed()):
            start = seg.end + offset_distance * seg.normal(1)
            end = next_seg.start + offset_distance * next_seg.normal(0)
            radius = offset_distance + offset_distance*1j
            rotation = 0
            large_arc = False
            sweep = True
            arc = Arc(start=start, radius=radius, rotation=rotation, large_arc=large_arc, sweep=sweep, end=end)
            offset_segments.extend([Line(offset_segments[-1].end,arc.start),arc])

    if path.isclosed():
        connect_the_dots.append(Line(nls[-1].end, nls[0].end))
    
    offset_path = Path(*offset_segments)
    return offset_path

# Examples:
seg1 = CubicBezier(230+200j, 100+100j, 200+200j, 200+300j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = Line(200+300j, 150+300j)  # A line beginning at (200, 300) and ending at (250, 350)
paths = Path(seg1,seg2)

distance = 10
offset_paths = offset_curve(paths, distance)

# Let's take a look
wsvg([paths, offset_paths], 'g' + 'r', filename='offset_curves.svg')

(CubicBezier(start=(230+200j), control1=(100+100j), control2=(200+200j), end=(200+300j)), Line(start=(200+300j), end=(150+300j)))
(Line(start=(200+300j), end=(150+300j)), CubicBezier(start=(230+200j), control1=(100+100j), control2=(200+200j), end=(200+300j)))


In [1]:
!code offset_curves.svg

Generate Character Paths

(0.9782376485029375-0.2074875973436566j)